# Main Analysis Pipeline

In [1]:
get_ipython().magic(u'run visualizations.ipynb')
import sys
from hits import CyDetHits, CTHHits, CDCHits, FlatHits
from memory_profiler import memory_usage
from pprint import pprint
from collections import Counter
sys.path.insert(0, '../modules')
from root_numpy import list_branches
%load_ext memory_profiler
from tracking import HoughSpace
from scipy import sparse
from tracking import HoughTransformer, HoughShifter
from cylinder import CyDet

Populating the interactive namespace from numpy and matplotlib


## Import the Signal Hits

In [2]:
def max_layer_cut(cdc_samp, max_layer):
    # Filter for max layer is five
    evt_max = np.zeros(cdc_samp.n_events)
    for event in range(cdc_samp.n_events):
        evt_hits = cdc_samp.get_measurement(cdc_samp.flat_name, events=event, 
                                            flatten=False,only_hits=False).astype(int)
        evt_max[event] = np.amax(cdc_samp.geom.point_layers[evt_hits])
    return np.unique(cdc_samp.get_events(events=np.where(evt_max >= max_layer))[cdc_samp.key_name])

In [3]:
def min_hits_cut(cdc_samp, min_hits):
    # Filter for number of signal hits
    good_events = np.where(cdc_samp.event_to_n_hits >= min_hits)[0]
    good_n_hits  = np.unique(cdc_samp.get_events(events=good_events)[cdc_samp.key_name])
    return good_n_hits

In [4]:
def import_cth_sample(cth_file):
    # Import the hits
    hits_cth = CTHHits(cth_sig_file, 
                       tree="CTHHitTree")
    # Get the trigger time
    hits_cth.set_trigger_time()
    return np.unique(hits_cth.data[hits_cth.get_trig_hits()][hits_cth.key_name])

In [5]:
def import_cdc_sample(cdc_file, min_hits=30, max_layer=4):
    # Import the hits
    cdc_samp = CyDetHits(cdc_file, 
                         tree="CDCHitTree",
                         selection="CDCHit.fIsSig == 1")
    # Get the events that pass the cuts
    enough_hits = min_hits_cut(cdc_samp, min_hits)
    print len(enough_hits)
    enough_layer = max_layer_cut(cdc_samp, max_layer)
    print len(enough_layer)
    return np.intersect1d(enough_hits, enough_layer)

## Import The Signal Hits

In [6]:
file_base = "/home/elg112/ICEDUST_old/track_finding_standalone/"\
            "track_finding_yandex/data/"
cdc_sig_file = file_base + "oa_xx_xxx_09000001-0000_7es54i4i543a_user-TrkTree_000_signal.root"
cth_sig_file = cdc_sig_file

In [7]:
good_cdc_events = import_cdc_sample(cdc_sig_file, 30, 4)
good_cth_events = import_cth_sample(cth_sig_file)

8267
7839


In [8]:
print good_cdc_events

[9000002 9000003 9000009 ..., 9019996 9019999 9020000]


In [9]:
print np.intersect1d(good_cdc_events, good_cth_events)

[9000003 9000015 9000016 ..., 9019988 9019996 9020000]


In [14]:
from rootpy.tree import Tree, TreeModel, TreeChain, BoolCol
from rootpy.io import root_open

class ExtraBranches(TreeModel):
    GoodTrack = BoolCol()
    GoodTrig = BoolCol()

def copy_in_trigger_signal(in_files_name, out_file_name, tree_name, 
                           prefix, cdc_events, cth_events):
    # Define the chain of input trees
    in_chain = TreeChain(name=tree_name, files=in_files_name)
    # First create a new file to save the new tree in:
    out_file = root_open(out_file_name, "recreate")
    out_tree = Tree(tree_name, model=ExtraBranches.prefix(prefix))

    # This creates all the same branches in the new tree but
    # their addresses point to the same memory used by the original tree.
    out_tree.create_branches(in_chain._buffer)
    out_tree.update_buffer(in_chain._buffer)
    
    # Now loop over the original tree(s) and fill the new tree
    for entry in in_chain:
        # Add in the new values
        out_tree.__setattr__(prefix+"GoodTrack", np.any(cdc_events == entry[prefix+"EventNumber"]))
        out_tree.__setattr__(prefix+"GoodTrig", np.any(cth_events == entry[prefix+"EventNumber"]))
        # Fill, noting that most of the buffer is shared between the chain 
        # and the output tree
        out_tree.Fill()
    # Close it up
    out_tree.Write()
    out_file.Close()

In [15]:
out_file_name = file_base+"with_trig_test.root"